# Підготовка та Аналіз даних
## Лабораторна робота №2
## Наука про дані: підготовчий етап
<b>Мета роботи</b>: ознайомитися з основними кроками по роботі з даними –
workflow від постановки задачі до написання пояснювальної записки,
зрозуміти постановку задачі та природу даних, над якими виконується
аналітичні операції
#### <b>Виконав:</b> <i>ФБ-35 Архипчук Віталій</i>

Перш за все виконаємо import потрібних біліотек для подальшої роботи:

In [2]:
import urllib.request
import pandas as pd
import os
from datetime import datetime

Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження.
Передбачити повторні запуски скрипту, довантаження нових даних та колізію даних;

In [4]:
def download_vhi( index ) :

    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={index}&year1=1981&year2=2024&type=Mean"
    
    now = datetime.now( )
    date_and_time = now.strftime( "%d-%m-%Y_%H-%M-%S" )
    
    os.makedirs( "VHI", exist_ok=True )
    
    filename = f'VHI/ID_{ index }_{ date_and_time }.csv'
    
    vhi_url = urllib.request.urlopen( url )
    with open( filename, 'wb' ) as out:
        out.write( vhi_url.read() )
    print( f"VHI для області {index} завантажено в {filename}" )

for i in range( 1, 28 ) :
    download_vhi( i )

VHI для області 1 завантажено в VHI/ID_1_15-03-2025_14-26-39.csv
VHI для області 2 завантажено в VHI/ID_2_15-03-2025_14-26-41.csv
VHI для області 3 завантажено в VHI/ID_3_15-03-2025_14-26-43.csv
VHI для області 4 завантажено в VHI/ID_4_15-03-2025_14-26-45.csv
VHI для області 5 завантажено в VHI/ID_5_15-03-2025_14-26-47.csv
VHI для області 6 завантажено в VHI/ID_6_15-03-2025_14-26-49.csv
VHI для області 7 завантажено в VHI/ID_7_15-03-2025_14-26-51.csv
VHI для області 8 завантажено в VHI/ID_8_15-03-2025_14-26-53.csv
VHI для області 9 завантажено в VHI/ID_9_15-03-2025_14-26-55.csv
VHI для області 10 завантажено в VHI/ID_10_15-03-2025_14-26-57.csv
VHI для області 11 завантажено в VHI/ID_11_15-03-2025_14-26-58.csv
VHI для області 12 завантажено в VHI/ID_12_15-03-2025_14-27-00.csv
VHI для області 13 завантажено в VHI/ID_13_15-03-2025_14-27-02.csv
VHI для області 14 завантажено в VHI/ID_14_15-03-2025_14-27-04.csv
VHI для області 15 завантажено в VHI/ID_15_15-03-2025_14-27-06.csv
VHI для облас

Зчитати завантажені текстові файли у фрейм
(https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)

(детальніше про роботу із фреймами буде розказано у подальших лабораторних роботах).

Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [3]:
def frame( path ) :

    headers = [ 'Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty' ]
    all_dataframes = [ ]
    
    for filename in os.listdir( path ) :
        if filename.endswith( '.csv' ) and filename.startswith( "ID" ):
            tokens = filename.split( "_" )
            index = int( tokens[1] )
            filepath = os.path.join( path, filename )

            df = pd.read_csv( filepath, header=1, names=headers, encoding='utf-8' )
            df = df[ df['VHI'] != -1 ]
            df[ 'Region' ] = index
            df[ 'Year' ] = df[ 'Year' ].astype( str ).str.replace( '<tt><pre>', '' ).str.replace( '</pre></tt>', '' ).str.strip( )
            df = df[ df['Year'] != '' ]
            df[ 'Year' ] = df[ 'Year' ].astype( int )
            df[ 'Week' ] = df[ 'Week' ].astype( int )
            df = df.drop( columns=['empty'] )

            all_dataframes.append( df )
    
    return pd.concat( all_dataframes, ignore_index=True ).drop_duplicates( ).reset_index( drop=True ) if all_dataframes else pd.DataFrame( )

folder_path = "VHI"
output_path = os.path.join(folder_path, "df.csv")

combined_dataframe = frame(folder_path)
combined_dataframe.to_csv(output_path, index=False, encoding='utf-8')

print( combined_dataframe )

       Year  Week    SMN     SMT    VCI    TCI    VHI  Region
0      1982     1  0.059  258.24  51.11  48.78  49.95      10
1      1982     2  0.063  261.53  55.89  38.20  47.04      10
2      1982     3  0.063  263.45  57.30  32.69  44.99      10
3      1982     4  0.061  265.10  53.96  28.62  41.29      10
4      1982     5  0.058  266.42  46.87  28.57  37.72      10
...     ...   ...    ...     ...    ...    ...    ...     ...
59017  2024    48  0.135  278.17  55.23  11.86  33.55       9
59018  2024    49  0.133  277.08  57.71  10.86  34.29       9
59019  2024    50  0.130  276.49  59.45   8.68  34.07       9
59020  2024    51  0.128  276.45  62.53   5.55  34.04       9
59021  2024    52  0.129  276.48  66.13   3.71  34.92       9

[59022 rows x 8 columns]


Реалізувати окрему процедуру, яка змінить індекси областей, які використані на порталі NOAA (за англійською абеткою) на наступні, за українською (виключно старі індекси на нові);

In [6]:
def replace_region( df ) :

    df = df.replace( {"Region": {
            24:1, 25:2, 5:3, 6:4, 27:5, 23:6, 26:7, 7:8, 11:9, 13:10, 
            14:11, 15:12, 16:13, 17:14, 18:15, 19:16, 21:17, 22:18, 8:19, 
            9:20, 10:21, 1:22, 3:23, 2:24, 4:25, 12:26, 20:27}
    } )
    
    return df

combined_dataframe = replace_region( combined_dataframe )

print( combined_dataframe )


       Year  Week    SMN     SMT    VCI    TCI    VHI  Region
0      1982   1.0  0.059  258.24  51.11  48.78  49.95      21
1      1982   2.0  0.063  261.53  55.89  38.20  47.04      21
2      1982   3.0  0.063  263.45  57.30  32.69  44.99      21
3      1982   4.0  0.061  265.10  53.96  28.62  41.29      21
4      1982   5.0  0.058  266.42  46.87  28.57  37.72      21
...     ...   ...    ...     ...    ...    ...    ...     ...
59017  2024  48.0  0.135  278.17  55.23  11.86  33.55      20
59018  2024  49.0  0.133  277.08  57.71  10.86  34.29      20
59019  2024  50.0  0.130  276.49  59.45   8.68  34.07      20
59020  2024  51.0  0.128  276.45  62.53   5.55  34.04      20
59021  2024  52.0  0.129  276.48  66.13   3.71  34.92      20

[59022 rows x 8 columns]


Реалізувати процедури для формування вибірок наступного виду
(включаючи елементи аналізу):

    1. Ряд VHI для області за вказаний рік;

In [7]:
def get_vhi( df, region, year ) :

    filtered_df = df[ (df['Region'] == region) & (df['Year'] == year) ]
    return filtered_df[ ['Year', 'Week', 'Region', 'VHI'] ]

region = 21
year = 1982

vhi_data = get_vhi( combined_dataframe, region, year )
print( vhi_data )

    Year  Week  Region    VHI
0   1982   1.0      21  49.95
1   1982   2.0      21  47.04
2   1982   3.0      21  44.99
3   1982   4.0      21  41.29
4   1982   5.0      21  37.72
5   1982   6.0      21  34.91
6   1982   7.0      21  33.14
7   1982   8.0      21  32.72
8   1982   9.0      21  32.77
9   1982  10.0      21  32.23
10  1982  11.0      21  30.38
11  1982  12.0      21  31.12
12  1982  13.0      21  31.65
13  1982  14.0      21  32.61
14  1982  15.0      21  35.49
15  1982  16.0      21  39.19
16  1982  17.0      21  41.14
17  1982  18.0      21  39.50
18  1982  19.0      21  37.07
19  1982  20.0      21  37.88
20  1982  21.0      21  40.99
21  1982  22.0      21  43.36
22  1982  23.0      21  45.31
23  1982  24.0      21  46.30
24  1982  25.0      21  48.85
25  1982  26.0      21  50.88
26  1982  27.0      21  51.83
27  1982  28.0      21  51.68
28  1982  29.0      21  51.61
29  1982  30.0      21  49.93
30  1982  31.0      21  46.00
31  1982  32.0      21  43.56
32  1982  

    2. Пошук екстремумів (min та max) для вказаних областей та років, середнього, медіани;

In [8]:
def analyze_extremes( df, regions, year ) :

    result = [ ]

    for region in regions :
        filtered_df = df[ (df['Region'] == region) & (df['Year'] == year) ]
        
        vhi_values = filtered_df[ 'VHI' ]
        
        min_vhi = vhi_values.min( )
        max_vhi = vhi_values.max( )
        mean_vhi = vhi_values.mean( )
        median_vhi = vhi_values.median( )
        
        result.append( {
            'Region': region,
            'Year': year,
            'Min VHI': min_vhi,
            'Max VHI': max_vhi,
            'Mean VHI': mean_vhi,
            'Median VHI': median_vhi
        } )
    
    result_df = pd.DataFrame( result )
    return result_df

regions = [ 21, 22 ]
year = 1982

extremes_df = analyze_extremes( combined_dataframe, regions, year )

print( extremes_df )


   Region  Year  Min VHI  Max VHI   Mean VHI  Median VHI
0      21  1982    22.82    51.83  37.359615       37.16
1      22  1982    23.83    64.58  43.835000       41.67


    3. Ряд VHI за вказаний діапазон років для вказаної області;

In [9]:
def analyze_vhi_range( df, region, start_year, end_year ) :

    filtered_df = df[ (df['Region'] == region) & (df['Year'].between(start_year, end_year)) ]
    
    vhi_values = filtered_df[ ['Year', 'VHI'] ]
    
    return vhi_values

region = 21
start_year = 1982
end_year = 1987

vhi_range_df = analyze_vhi_range( combined_dataframe, region, start_year, end_year )

print( vhi_range_df )

     Year    VHI
0    1982  49.95
1    1982  47.04
2    1982  44.99
3    1982  41.29
4    1982  37.72
..    ...    ...
296  1987  31.89
297  1987  29.87
298  1987  27.96
299  1987  27.21
300  1987  29.46

[301 rows x 2 columns]


    4. Для всього набору даних виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25). Повернути роки, назви областей з екстремальними посухами та значення VHI;

In [17]:
def extreme_droughts( df ) :
    
    drought_df = df[ df['VHI'] < 15 ]

    drought_counts = drought_df.groupby( 'Year' )[ 'Region' ].nunique( )

    years_with_extreme_droughts = drought_counts[ drought_counts >= 5 ].index

    result_df = drought_df[ drought_df['Year'].isin(years_with_extreme_droughts) ]
    
    print( "Роки, коли екстремальні посухи торкалися більше ніж 20% областей:" )
    
    for year in years_with_extreme_droughts :
        affected_regions = result_df[ result_df['Year'] == year ]
        print( f"\nРік: {year}" )
        print( f"Кількість областей з екстремальними посухами: {affected_regions['Region'].nunique()}" )
        print( f"Області з екстремальними посухами та значення VHI:" )

        for _, row in affected_regions.iterrows( ) :
            print( f"  Область {row['Region']}: Week {row['Week']} VHI = {row['VHI']}" )
    
    return

extreme_droughts_df = extreme_droughts( combined_dataframe )

print( extreme_droughts_df )

Роки, коли екстремальні посухи торкалися більше ніж 20% областей:

Рік: 2000
Кількість областей з екстремальними посухами: 6
Області з екстремальними посухами та значення VHI:
  Область 9.0: Week 44.0 VHI = 12.51
  Область 9.0: Week 45.0 VHI = 10.6
  Область 9.0: Week 46.0 VHI = 11.2
  Область 9.0: Week 47.0 VHI = 12.32
  Область 9.0: Week 48.0 VHI = 14.65
  Область 26.0: Week 42.0 VHI = 14.89
  Область 26.0: Week 43.0 VHI = 12.76
  Область 26.0: Week 44.0 VHI = 7.81
  Область 26.0: Week 45.0 VHI = 6.49
  Область 26.0: Week 46.0 VHI = 6.58
  Область 26.0: Week 47.0 VHI = 6.71
  Область 26.0: Week 48.0 VHI = 7.56
  Область 26.0: Week 49.0 VHI = 9.25
  Область 26.0: Week 50.0 VHI = 10.94
  Область 26.0: Week 51.0 VHI = 12.28
  Область 22.0: Week 44.0 VHI = 14.64
  Область 22.0: Week 45.0 VHI = 11.82
  Область 22.0: Week 46.0 VHI = 10.81
  Область 22.0: Week 47.0 VHI = 10.68
  Область 22.0: Week 48.0 VHI = 12.3
  Область 22.0: Week 49.0 VHI = 14.24
  Область 27.0: Week 45.0 VHI = 13.14
  